In [5]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

BATCH_SIZE = 224
EPOCH = 120

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

#transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomCrop(170),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.RandomCrop(170),
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

train_dataset = ImageFolder(root='../dataset/train', transform=data_transforms['train'])
val_dataset = ImageFolder(root='../dataset/valid', transform=data_transforms['valid'])

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

cuda


In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 450)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [9]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader) 
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train. Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('valid. Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train. Loss: 4.6662, Accuracy: 8.08%
valid. Loss: 4.5492, Accuracy: 8.22%
Completed in 1m 11s
-------------- epoch 2 ----------------
train. Loss: 4.1204, Accuracy: 16.63%
valid. Loss: 4.0037, Accuracy: 16.71%
Completed in 1m 11s
-------------- epoch 3 ----------------
train. Loss: 3.6776, Accuracy: 22.84%
valid. Loss: 3.5325, Accuracy: 24.98%
Completed in 1m 19s
-------------- epoch 4 ----------------
train. Loss: 3.4459, Accuracy: 27.42%
valid. Loss: 3.3161, Accuracy: 27.87%
Completed in 1m 12s
-------------- epoch 5 ----------------
train. Loss: 3.2610, Accuracy: 29.80%
valid. Loss: 3.0490, Accuracy: 33.24%
Completed in 1m 15s
-------------- epoch 6 ----------------
train. Loss: 3.0174, Accuracy: 34.32%
valid. Loss: 2.8463, Accuracy: 36.09%
Completed in 1m 14s
-------------- epoch 7 ----------------
train. Loss: 2.9668, Accuracy: 36.05%
valid. Loss: 2.7582, Accuracy: 38.49%
Completed in 1m 11s
-------------- epoch 8 ----------------
train. Los

-------------- epoch 62 ----------------
train. Loss: 1.8108, Accuracy: 57.44%
valid. Loss: 1.6605, Accuracy: 61.07%
Completed in 1m 10s
-------------- epoch 63 ----------------
train. Loss: 1.8781, Accuracy: 56.12%
valid. Loss: 1.7380, Accuracy: 59.16%
Completed in 1m 8s
-------------- epoch 64 ----------------
train. Loss: 1.7644, Accuracy: 58.46%
valid. Loss: 1.6620, Accuracy: 60.44%
Completed in 1m 10s
-------------- epoch 65 ----------------
train. Loss: 1.7657, Accuracy: 58.43%
valid. Loss: 1.6507, Accuracy: 59.20%
Completed in 1m 12s
-------------- epoch 66 ----------------
train. Loss: 1.8321, Accuracy: 56.74%
valid. Loss: 1.6684, Accuracy: 59.60%
Completed in 1m 7s
-------------- epoch 67 ----------------
train. Loss: 1.7974, Accuracy: 57.45%
valid. Loss: 1.7005, Accuracy: 59.33%
Completed in 1m 7s
-------------- epoch 68 ----------------
train. Loss: 1.7556, Accuracy: 58.29%
valid. Loss: 1.6339, Accuracy: 59.38%
Completed in 1m 8s
-------------- epoch 69 ----------------
trai